In [1]:
#| default_exp components

# Components

In [2]:
#| export
from html.parser import HTMLParser
from dataclasses import dataclass, asdict

from fastcore.utils import *
from fastcore.xml import *
from fastcore.meta import use_kwargs, delegates

try: from IPython import display
except ImportError: display=None

In [47]:
from pprint import pprint
from lxml import html as lx

In [4]:
#| export
def show(xt,*rest):
    if rest: xt = (xt,)+rest
    return display.HTML(to_xml(xt))

In [5]:
#| export
picocss = "https://cdn.jsdelivr.net/npm/@picocss/pico@latest/css/pico.min.css"
picolink = Link(rel="stylesheet", href=picocss)
picocondcss = "https://cdn.jsdelivr.net/npm/@picocss/pico@latest/css/pico.conditional.min.css"
picocondlink = Link(rel="stylesheet", href=picocondcss)

`picocondlink` is the class-conditional css `link` tag, and `picolink` is the regular tag.

In [6]:
show(picocondlink)

In [7]:
#| export
def set_pico_cls():
    js = """var sel = '.cell-output, .output_area';
document.querySelectorAll(sel).forEach(e => e.classList.add('pico'));

new MutationObserver(ms => {
  ms.forEach(m => {
    m.addedNodes.forEach(n => {
      if (n.nodeType === 1) {
        var nc = n.classList;
        if (nc && (nc.contains('cell-output') || nc.contains('output_area'))) {
          nc.add('pico');
        }
        n.querySelectorAll(sel).forEach(e => e.classList.add('pico'));
      }
    });
  });
}).observe(document.body, { childList: true, subtree: true });"""
    return display.Javascript(js)

Run this to make jupyter outputs styled with pico:

In [8]:
set_pico_cls()

<IPython.core.display.Javascript object>

In [9]:
#| export
named = set('a button form frame iframe img input map meta object param select textarea'.split())
html_attrs = 'id cls title style'.split()
hx_attrs = 'get post put delete patch trigger target swap include select indicator push_url confirm disable replace_url on'
hx_attrs = html_attrs + [f'hx_{o}' for o in hx_attrs.split()]

In [10]:
#| export
@use_kwargs(html_attrs, keep=True)
def xt_html(tag: str, *c, **kwargs):
    tag,c,kw = xt(tag, *c, **kwargs)
    if tag in named and 'id' in kw and 'name' not in kw: kw['name'] = kw['id']
    return XT([tag,c,kw])

In [11]:
#| export
@use_kwargs(hx_attrs, keep=True)
def xt_hx(tag: str, *c, target_id=None, **kwargs):
    if target_id: kwargs['hx_target'] = '#'+target_id
    return xt_html(tag, *c, **kwargs)

In [12]:
#| export
_g = globals()
_all_ = ['Html', 'Head', 'Title', 'Meta', 'Link', 'Style', 'Body', 'Pre', 'Code',
    'Div', 'Span', 'P', 'H1', 'H2', 'H3', 'H4', 'H5', 'H6', 'Strong', 'Em', 'B',
    'I', 'U', 'S', 'Strike', 'Sub', 'Sup', 'Hr', 'Br', 'Img', 'Link', 'Nav',
    'Ul', 'Ol', 'Li', 'Dl', 'Dt', 'Dd', 'Table', 'Thead', 'Tbody', 'Tfoot', 'Tr',
    'Th', 'Td', 'Caption', 'Col', 'Colgroup', 'Form', 'Input', 'Textarea',
    'Button', 'Select', 'Option', 'Label', 'Fieldset', 'Legend', 'Details', 'Dialog',
    'Summary', 'Main', 'Header', 'Footer', 'Section', 'Article', 'Aside', 'Figure',
    'Figcaption', 'Mark', 'Small', 'Iframe', 'Object', 'Embed', 'Param', 'Video',
    'Audio', 'Source', 'Canvas', 'Svg', 'Math', 'Script', 'Noscript', 'Template', 'Slot']

for o in _all_: _g[o] = partial(xt_hx, o.lower())

For tags that have a `name` attribute, it will be set to the value of `id` if not provided explicitly:

In [13]:
Form(Button(target_id='foo', id='btn'),
     hx_post='/', target_id='tgt', id='frm')

['form',
 (['button', (), {'id': 'btn', 'hx-target': '#foo', 'name': 'btn'}],),
 {'hx-post': '/', 'id': 'frm', 'hx-target': '#tgt', 'name': 'frm'}]

In [14]:
#| export
@delegates(xt_hx, keep=True)
def A(*c, hx_get=None, target_id=None, hx_swap=None, href='#', **kwargs):
    return xt_hx('a', *c, href=href, hx_get=hx_get, target_id=target_id, hx_swap=hx_swap, **kwargs)

In [15]:
A('text', ht_get='/get', target_id='id')

['a', ('text',), {'href': '#', 'ht-get': '/get', 'hx-target': '#id'}]

In [16]:
#| export
@delegates(xt_hx, keep=True)
def AX(txt, hx_get=None, target_id=None, hx_swap=None, href='#', **kwargs):
    return xt_hx('a', txt, href=href, hx_get=hx_get, target_id=target_id, hx_swap=hx_swap, **kwargs)

In [17]:
AX('text', '/get', 'id')

['a', ('text',), {'href': '#', 'hx-get': '/get', 'hx-target': '#id'}]

In [18]:
#| export
@delegates(xt_hx, keep=True)
def Checkbox(checked:bool=False, label=None, **kwargs):
    if not checked: checked=None
    res = Input(type="checkbox", checked=checked, **kwargs)
    if label: res = Label(res, label)
    return res

In [19]:
show(Checkbox(True, 'Check me out!'))

In [20]:
#| export
@delegates(xt_hx, keep=True)
def Card(*c, header=None, footer=None, **kwargs):
    if header: c = (Header(header),) + c
    if footer: c += (Footer(footer),)
    return Article(*c, **kwargs)

In [21]:
show(Card('body', header=P('head'), footer=P('foot')))

In [22]:
#| export
@delegates(xt_hx, keep=True)
def Group(*c, **kwargs):
    return Fieldset(*c, role="group", **kwargs)

In [23]:
show(Group(Input(), Button("Save")))

In [24]:
#| export
@delegates(xt_hx, keep=True)
def Search(*c, **kwargs):
    return Form(*c, role="search", **kwargs)

In [25]:
show(Search(Input(type="search"), Button("Search")))

In [26]:
#| export
@delegates(xt_hx, keep=True)
def Grid(*c, cls='grid', **kwargs):
    c = tuple(o if isinstance(o,list) else Div(o) for o in c)
    return xt_hx('div', *c, cls=cls, **kwargs)

In [27]:
colors = [Input(type="color", value=o) for o in ('#e66465', '#53d2c5', '#f6b73c')]
show(Grid(*colors))

In [28]:
#| export
@delegates(xt_hx, keep=True)
def DialogX(*c, open=None, header=None, footer=None, id=None, **kwargs):
    card = Card(*c, header=header, footer=footer, **kwargs)
    return Dialog(card, open=open, id=id)

In [29]:
hdr = Div(Button(aria_label="Close", rel="prev"), P('confirm'))
ftr = Div(Button('Cancel', cls="secondary"), Button('Confirm'))
d = DialogX('thank you!', header=hdr, footer=ftr, open=None, id='dlgtest')
# use js or htmx to display modal

In [30]:
#| export
@delegates(xt_hx, keep=True)
def Hidden(value:str="", **kwargs):
    return Input(type="hidden", value=value, **kwargs)

In [31]:
#| export
def set_val(tag, attr, val):
    if attr.get('type', '') in ('checkbox','radio'):
        if val: attr['checked'] = '1'
        else: attr.pop('checked', '')
    else: attr['value'] = val

In [32]:
#| export
def find_inps(html):
    if not html: return []
    tag,cs,attrs = html
    if tag == 'input': return [html]
    res = []
    for c in cs:
        if isinstance(c, list): res.extend(find_inps(c))
    return res

In [33]:
#| export
def fill_form(form, obj):
    "Modifies form in-place and returns it"
    inps = find_inps(form)
    inps = {attrs['id']:(tag,attrs) for tag,c,attrs in inps if 'id' in attrs}
    for nm,val in asdict(obj).items():
        if nm in inps:
            tag,attr = inps[nm]
            set_val(tag, attr, val)
    return form

In [34]:
@dataclass
class TodoItem:
    title:str; id:int; done:bool
                
todo = TodoItem(id=2, title="Profit", done=True)
check = Checkbox(id="done", label='Done')
form = Form(Fieldset(Input(id="title"), check, Hidden(id="id"), Button("Save")))
fill_form(form, todo)

['form',
 (['fieldset',
   (['input', (), {'id': 'title', 'name': 'title', 'value': 'Profit'}],
    ['label',
     (['input',
       (),
       {'type': 'checkbox', 'id': 'done', 'name': 'done', 'checked': '1'}],
      'Done'),
     {}],
    ['input', (), {'type': 'hidden', 'value': 2, 'id': 'id', 'name': 'id'}],
    ['button', ('Save',), {}]),
   {}],),
 {}]

In [35]:
#|export
def fill_dataclass(src, dest):
    "Modifies dataclass in-place and returns it"
    for nm,val in asdict(src).items(): setattr(dest, nm, val)
    return dest

In [36]:
nt = TodoItem('', 0, False)
fill_dataclass(todo, nt)
nt

TodoItem(title='Profit', id=2, done=True)

In [37]:
#| export
class _FindElems(HTMLParser):
    def __init__(self, tag=None, attr=None, **props):
        super().__init__()
        self.tag,self.attr,self.props = tag,attr,props
        self.res = []

    def handle_starttag(self, tag, attrs):
        if self.tag and tag!=self.tag: return
        d = dict(attrs)
        if [k for k,v in self.props.items() if d.get(k,None)==v]:
            self.res.append(d.get(self.attr, None) if self.attr else d)

In [38]:
#| export
def find_elems(s:XT|str, tag=None, attr=None, **props):
    "Find elements in `s` with `tag` (if supplied) and `props`, returning `attr`"
    o = _FindElems(tag, attr, **props)
    o.feed(to_xml(s))
    return o.res

In [39]:
find_elems(form, 'input', 'value', id='title')

['Profit']

In [49]:
elem = lx.fromstring(to_xml(form))
elem.xpath("//input[@id='title']/@value")

['Profit']

# Export -

In [50]:
#|hide
import nbdev; nbdev.nbdev_export()